In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import Row

conf = SparkConf().setMaster("local").setAppName("Project_BigData")
sc = SparkContext.getOrCreate(conf = conf)

In [2]:
#Read the data
h1b_data = sc.textFile(\
           "hdfs://quickstart.cloudera:8020/user/cloudera/Project/H-1B_Disclosure_Data_FY17.csv")
h1b_data.take(5)

[u',CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_REPRESENTING_EMPLOYER,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAICS_CODE,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_LEVEL,PW_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE',
 u'0,I-200-16055-173457,CERTIFIED-WITHDRAWN,2016-02-24,2016-10-01,H-1B,2016-08-10,2019-08-10,DISCOVER PRODUCTS INC.,,

In [3]:
def createRow(data, headers):
    data_dict = {}
    data_list = data.split(",")
    for i in range(len(headers)):
        if(headers[i] != ""):
            data_dict[headers[i]] = data_list[i]
    #print(data_dict)
    return data_dict

In [4]:
#Get the headers
headers_string = h1b_data.take(1)[0]
headers = headers_string.split(",")
headers

[u'',
 u'CASE_NUMBER',
 u'CASE_STATUS',
 u'CASE_SUBMITTED',
 u'DECISION_DATE',
 u'VISA_CLASS',
 u'EMPLOYMENT_START_DATE',
 u'EMPLOYMENT_END_DATE',
 u'EMPLOYER_NAME',
 u'EMPLOYER_BUSINESS_DBA',
 u'EMPLOYER_ADDRESS',
 u'EMPLOYER_CITY',
 u'EMPLOYER_STATE',
 u'EMPLOYER_POSTAL_CODE',
 u'EMPLOYER_COUNTRY',
 u'EMPLOYER_PROVINCE',
 u'EMPLOYER_PHONE',
 u'EMPLOYER_PHONE_EXT',
 u'AGENT_REPRESENTING_EMPLOYER',
 u'AGENT_ATTORNEY_NAME',
 u'AGENT_ATTORNEY_CITY',
 u'AGENT_ATTORNEY_STATE',
 u'JOB_TITLE',
 u'SOC_CODE',
 u'SOC_NAME',
 u'NAICS_CODE',
 u'TOTAL_WORKERS',
 u'NEW_EMPLOYMENT',
 u'CONTINUED_EMPLOYMENT',
 u'CHANGE_PREVIOUS_EMPLOYMENT',
 u'NEW_CONCURRENT_EMPLOYMENT',
 u'CHANGE_EMPLOYER',
 u'AMENDED_PETITION',
 u'FULL_TIME_POSITION',
 u'PREVAILING_WAGE',
 u'PW_UNIT_OF_PAY',
 u'PW_WAGE_LEVEL',
 u'PW_SOURCE',
 u'PW_SOURCE_YEAR',
 u'PW_SOURCE_OTHER',
 u'WAGE_RATE_OF_PAY_FROM',
 u'WAGE_RATE_OF_PAY_TO',
 u'WAGE_UNIT_OF_PAY',
 u'H1B_DEPENDENT',
 u'WILLFUL_VIOLATOR',
 u'SUPPORT_H1B',
 u'LABOR_CON_AGREE',

In [5]:
#Remove the header from the data
h1b_data = h1b_data.filter(lambda x: x != headers_string)
h1b_data.take(1)

[u'0,I-200-16055-173457,CERTIFIED-WITHDRAWN,2016-02-24,2016-10-01,H-1B,2016-08-10,2019-08-10,DISCOVER PRODUCTS INC.,,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,,2244050900,,Y,"ELLSWORTH, CHAD",NEW YORK,NY,ASSOCIATE DATA INTEGRATION,15-1121,COMPUTER SYSTEMS ANALYSTS,522210,1,1,0,0,0,0,0,Y,59197.0,Year,Level I,OES,2015.0,OFLC ONLINE DATA CENTER,65811.0,67320.0,Year,N,N,,Y,,RIVERWOODS,LAKE,IL,60015,2016-03-01']

In [6]:
h1b_data_frame = h1b_data.map(lambda x: Row(**createRow(x, headers))).toDF()
h1b_data_frame.take(1)

[Row(AGENT_ATTORNEY_CITY=u' CHAD"', AGENT_ATTORNEY_NAME=u'"ELLSWORTH', AGENT_ATTORNEY_STATE=u'NEW YORK', AGENT_REPRESENTING_EMPLOYER=u'Y', AMENDED_PETITION=u'0', CASE_NUMBER=u'I-200-16055-173457', CASE_STATUS=u'CERTIFIED-WITHDRAWN', CASE_SUBMITTED=u'2016-02-24', CHANGE_EMPLOYER=u'0', CHANGE_PREVIOUS_EMPLOYMENT=u'0', CONTINUED_EMPLOYMENT=u'1', DECISION_DATE=u'2016-10-01', EMPLOYER_ADDRESS=u'2500 LAKE COOK ROAD', EMPLOYER_BUSINESS_DBA=u'', EMPLOYER_CITY=u'RIVERWOODS', EMPLOYER_COUNTRY=u'UNITED STATES OF AMERICA', EMPLOYER_NAME=u'DISCOVER PRODUCTS INC.', EMPLOYER_PHONE=u'2244050900', EMPLOYER_PHONE_EXT=u'', EMPLOYER_POSTAL_CODE=u'60015', EMPLOYER_PROVINCE=u'', EMPLOYER_STATE=u'IL', EMPLOYMENT_END_DATE=u'2019-08-10', EMPLOYMENT_START_DATE=u'2016-08-10', FULL_TIME_POSITION=u'0', H1B_DEPENDENT=u'Year', JOB_TITLE=u'NY', LABOR_CON_AGREE=u'', NAICS_CODE=u'COMPUTER SYSTEMS ANALYSTS', NEW_CONCURRENT_EMPLOYMENT=u'0', NEW_EMPLOYMENT=u'1', ORIGINAL_CERT_DATE=u'60015', PREVAILING_WAGE=u'Y', PUBLIC_DI

In [8]:
h1b_data_frame[h1b_data_frame.AGENT_REPRESENTING_EMPLOYER == "Y"].show()

+-------------------+-------------------+--------------------+---------------------------+----------------+------------------+-------------------+--------------+---------------+--------------------------+--------------------+-------------+--------------------+---------------------+-------------+--------------------+--------------------+--------------+------------------+--------------------+-----------------+--------------+-------------------+---------------------+------------------+-------------+---------+---------------+--------------------+-------------------------+--------------+------------------+---------------+--------------------------+---------+---------------+--------------+--------------+-------------+--------------------+--------+-----------+-----------------+----------+---------------------+--------------------+----------------+----------------+-------------+---------------+--------------------+--------------+
|AGENT_ATTORNEY_CITY|AGENT_ATTORNEY_NAME|AGENT_ATTORNEY_STATE|AG